In [11]:
import traffic
import pandas as pd 
import os
from traffic.core import Traffic
from biblio_herramienta.herramienta import *
from biblio_herramienta.tratardatos import *
from biblio_herramienta.ml import *

In [12]:
archivoBBDD = r"/Users/jaimebowen/OneDrive/TFGSeptiembre/libros/datos_sectores/datos_buenos/BBDD2_filtrado.csv"

In [13]:
bbdd = pd.read_csv(archivoBBDD)

In [14]:
bbdd.head()

,Unnamed: 0,altitude_1,geoaltitude_1,latitude_1,longitude_1,track_unwrapped_1,cluster_1,track_unwrapped_2,cluster_2,Init separation,Init acimut,Var GS Module,Var Track,Var init altitude,Var Vertical speed,Conflicto,MinDis,Timetoconf
0,0,35025.0,35000.0,44.375129,-3.97484,187.314737,3,23.372108,6,104.868237,-156.321253,912.002645,163.942628,-975.0,-64.0,0,10.0,10000
1,1,35025.0,35000.0,44.375129,-3.97484,187.314737,3,183.321476,3,1.104216,89.266358,50.057158,3.993261,25.0,-64.0,0,10.0,10000
2,2,35025.0,35000.0,44.375129,-3.97484,187.314737,3,22.853767,4,130.363624,159.469960,893.772561,164.460969,-975.0,-64.0,0,10.0,10000
3,3,35025.0,35000.0,44.375129,-3.97484,187.314737,3,187.867927,3,0.155850,-101.368558,85.107283,-0.553190,-3975.0,-64.0,0,10.0,10000
4,4,35025.0,35000.0,44.375129,-3.97484,187.314737,3,12.617341,4,125.026066,164.574815,907.033171,174.697396,500.0,-960.0,0,10.0,10000


In [15]:
bbdd.drop(columns = ["Unnamed: 0"],axis = 1, inplace = True)
bbdd.head()

,altitude_1,geoaltitude_1,latitude_1,longitude_1,track_unwrapped_1,cluster_1,track_unwrapped_2,cluster_2,Init separation,Init acimut,Var GS Module,Var Track,Var init altitude,Var Vertical speed,Conflicto,MinDis,Timetoconf
0,35025.0,35000.0,44.375129,-3.97484,187.314737,3,23.372108,6,104.868237,-156.321253,912.002645,163.942628,-975.0,-64.0,0,10.0,10000
1,35025.0,35000.0,44.375129,-3.97484,187.314737,3,183.321476,3,1.104216,89.266358,50.057158,3.993261,25.0,-64.0,0,10.0,10000
2,35025.0,35000.0,44.375129,-3.97484,187.314737,3,22.853767,4,130.363624,159.469960,893.772561,164.460969,-975.0,-64.0,0,10.0,10000
3,35025.0,35000.0,44.375129,-3.97484,187.314737,3,187.867927,3,0.155850,-101.368558,85.107283,-0.553190,-3975.0,-64.0,0,10.0,10000
4,35025.0,35000.0,44.375129,-3.97484,187.314737,3,12.617341,4,125.026066,164.574815,907.033171,174.697396,500.0,-960.0,0,10.0,10000


# 2. Generación de los datatsets
Con los datos cargados, ahora se van a generar dos BBDD:
* La que se va a usar para entrenar al modelo
* La que se va a usar para saber si el modelo ha sido bien entrenado

In [17]:
X = bbdd.drop(columns = ["Conflicto"],axis =1)
y = bbdd["Conflicto"]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X , y , stratify = y, test_size = 0.2, random_state = 42)

# 3. Generación de Pipelines 
Como se comenta en el apartado 2 el TFG, se va a usar varios método para realizar las predicciones de los datos obtenidos y probar cual es aquel que tiene mayor precisión.  


## 3.1 Modelos lineales
### 3.1.1 Clasificadro Ridge

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeClassifier

# Clasificador Ridge
ridge_pipe = Pipeline([
    ('scaler',StandardScaler()),
    ('ridge_classifier',RidgeClassifier(alpha = 1))


])

# Clasificador 

In [20]:
ridge_pipe.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('ridge_classifier', RidgeClassifier(alpha=1))])

In [22]:
ridge_pipe.score(X_test,y_test)

1.0

### 3.1.1 Clasificadro SVM

In [25]:
from sklearn.svm import LinearSVC
svm_pipe = Pipeline((
    ('scaler', StandardScaler()),
    ('linear_svc', LinearSVC(C=1, loss = 'hinge'))
))

In [26]:
svm_pipe.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=1, loss='hinge'))])

In [27]:
svm_pipe.score(X_test,y_test)

SyntaxError: invalid syntax (<ipython-input-27-ca9043000fe2>, line 1)